In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key

In [2]:
city_data_df=pd.read_csv("output_data/cities.csv")
city_data_df.head()

,Citi_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mataura,-46.1927,168.8643,47.48,98,100,3.51,NZ,1659022958
1,1,Belmonte,-15.8631,-38.8828,77.34,58,41,4.79,BR,1659022958
2,2,Saint Paul Harbor,57.7900,-152.4072,49.51,66,0,6.91,US,1659022959
3,3,Tuktoyaktuk,69.4541,-133.0374,46.40,81,100,10.36,CA,1659022959
4,4,Caravelas,-17.7125,-39.2481,78.42,55,18,7.52,BR,1659022959


In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
#heatmap of the humidity
locations=city_data_df[["Lat","Lng"]]
humidity=city_data_df["Humidity"]
fig=gmaps.figure()
heat_layer=gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=300,point_radius=5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
idealweatherdf=city_data_df.loc[
    (city_data_df["Max Temp"]>70)&(city_data_df["Max Temp"]<80)&\
    (city_data_df["Wind Speed"]>10)&\
    (city_data_df["Cloudiness"]==0)
    
].dropna()
idealweatherdf.head()

,Citi_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,Safi,32.2994,-9.2372,74.44,71,0,17.58,MA,1659022961
351,351,Coralville,41.6764,-91.5805,75.43,63,0,11.50,US,1659023078
399,399,Menomonie,44.8755,-91.9193,70.12,49,0,14.27,US,1659023093
500,500,Elk Point,42.6833,-96.6836,72.90,29,0,13.22,US,1659023131


In [8]:
hotel_df=idealweatherdf[["City","Country","Lat","Lng"]]
hotel_df["Hotel Name"]=""
hotel_df

C:\Users\mohamed\AppData\Local\Temp\ipykernel_4920\4157048328.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=""


,City,Country,Lat,Lng,Hotel Name
11,Safi,MA,32.2994,-9.2372,
351,Coralville,US,41.6764,-91.5805,
399,Menomonie,US,44.8755,-91.9193,
500,Elk Point,US,42.6833,-96.6836,


In [10]:
params={
    "radius":5000,
    "types":"lodging",
    "key": g_key
}
for index,row in hotel_df.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"]=f"{lat},{lng}"
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address=requests.get(base_url,params=params).json()
    try:
        hotel_df.loc[index,"Hotel Name"]=name_address["results"][0]["name"]
    except:
        print(f"Error finding lodging in {row['City']}. Skipping")
        pass
hotel_df

,City,Country,Lat,Lng,Hotel Name
11,Safi,MA,32.2994,-9.2372,Hôtel Abda
351,Coralville,US,41.6764,-91.5805,Heartland Inn
399,Menomonie,US,44.8755,-91.9193,Best Western Plus Menomonie Inn & Suites
500,Elk Point,US,42.6833,-96.6836,Hometowne Inn


In [11]:
info_box_template="""
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info=[info_box_template.format(**row) for index,row in hotel_df.iterrows()]
locations=hotel_df[["Lat","Lng"]]

In [12]:
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))